In [18]:
#Functional Goal: time step the mechanics n times 
def time_step(x,X,h):
    '''TODO: define the simplest productive time step 
    - unbiased R.W. that only accepts local improvements.
    - 1. FEI directed by elastic model.
    - 2. FEI directed by spring model.
    - 3. Newmark directed by elastic model.
    - 4. Newmark directed by spring model.
    - R.W. that only accepts local improvements, with steps directed spring/elastic model.
    - '''
    return 0.
def time_step_n_times(x,X,h,n):
    for k in range(n):
        x += time_step(x,X,h)
    return x
def get_h(h = 10**-2):
    '''constant time steps'''
    return h
def get_h(h = 10**-2, beta = 1., acceptedQ=True):
    '''exponential adaptive time steps
    h = most recent time step,
    acceptedQ = whether the most recent time step was accepted,
    beta = stepsize change in parameter.
    **caution** this get_h could be unstable/inefficient with any unbiased R.W..
    Use it with steps directed towards the minimizer
    '''
    return h

mesh.vertices = time_step_n_times(
    mesh.vertices,
    X,
    get_h(),
    n=1)

# Development of Newmark Time Stepping
Tim Tyree<br>
7.16.2020

In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  

#automate the boring stuff
from IPython import utils
import time, os, sys, re
beep = lambda x: os.system("echo -n '\\a';sleep 0.2;" * x)
if not 'nb_dir' in globals():
    nb_dir = os.getcwd()
sys.path.append("../lib") 
from lib import *
sys.path.append("lib") 
from lib import *

# from operari import *
# from ProgressBar import *
# from mesh_ops import *

# the visualization tools involved here for triangular meshes is
import trimesh
import pyglet
from numba import njit, cuda
# from numba.typed import List
# import numba
import trimesh

#try using a scipy sparse matrix to speed up spring force evaluations
#TODO: speed up bigger meshes with pycuda's sparce matrices
from scipy.sparse import csr_matrix
import scipy.sparse as sparse



# from pyspark import SparkContext 
# sc = SparkContext(master="local[4]")
# print(sc)


%autocall 1
%load_ext autoreload
%autoreload 2

Automatic calling is: Smart


## For a Single Point With Net Input Forces Given, in Numpy

In [5]:
import numpy as np
from numpy import zeros,dot,atleast_1d
from scipy import linalg

In [7]:
# adapted from the following incumbent
##########################################################################
# program: Newmark.py
# author: Tom Irvine
# version: 1.4
# date: May 1, 2012
# description:  solution of a system of second-order ODEs for a dynamic
#               system via the Newmark-beta method
#
##########################################################################

In [8]:
def Newmark_coefficients(dt):
    alpha=0.25
    beta=0.5

    a0=1/(alpha*(dt**2))
    a1=beta/(alpha*dt)
    a2=1/(alpha*dt)
    a3=(1/(2*alpha))-1
    a4=(beta/alpha)-1
    a5=(dt/2)*((beta/alpha)-2)
    a6=dt*(1-beta)
    a7=beta*dt

    return a0,a1,a2,a3,a4,a5,a6,a7

In [15]:
Newmark_coefficients(dt=.05)

(1599.9999999999998, 40.0, 80.0, 1.0, 1.0, 0.0, 0.025, 0.025)

In [9]:
def Newmark_initialize(ndof,a0,a1,M,C,K,NT,DI,VI):

    nlength=len(atleast_1d(M)) 

    KH=zeros((ndof,ndof),float)

    KH=K+a0*M+a1*C

    if(nlength==1):

        U=zeros(NT,float)
        Ud=zeros(NT,float)
        Udd=zeros(NT,float)

        U[0]=DI
        Ud[0]=VI
            
    else:

        U=zeros((ndof,NT),float)
        Ud=zeros((ndof,NT),float)
        Udd=zeros((ndof,NT),float)

        U[:,0]=DI
        Ud[:,0]=VI

    return U,Ud,Udd,KH

In [10]:
def Newmark_force(M,C,K,FFI,force_dof,VI,DI,dt,NT,ndof):
    """
    input

    M = mass matrix
    C = damping matrix
    K = stiffness matrix

    FFI = interpolated force matrix
    force_dof = connects forces with dofs

    VI = initial velocity
    DI = initial displacement

      dt = time step
      NT = number of time points
    ndof = number of degrees of freedom

    output

      U = displacement
     Ud = velocity
    Udd = acceleration

    """

    a0,a1,a2,a3,a4,a5,a6,a7=Newmark_coefficients(dt)

    U,Ud,Udd,KH=Newmark_initialize(ndof,a0,a1,M,C,K,NT,DI,VI)


    for i in range (1,NT):

        V1=(a1*U[:,i-1]+a4*Ud[:,i-1]+a5*Udd[:,i-1])
        V2=(a0*U[:,i-1]+a2*Ud[:,i-1]+a3*Udd[:,i-1])

        CV=dot(C,V1)
        MA=dot(M,V2)


#  apply forces

        F=zeros(ndof,float)

        for j in range (0,ndof):

            j_index=force_dof[j]

            if(j_index!=-999):

                F[j]=FFI[i,j_index]


        FH=F+MA+CV

#  solve for displacements

        if(ndof>1):
            Un = linalg.solve(KH, FH)
        else:
            Un=FH/KH

        Uddn=a0*(Un-U[:,i-1])-a2*Ud[:,i-1]-a3*Udd[:,i-1]
        Udn=Ud[:,i-1]+a6*Udd[:,i-1]+a7*Uddn


        U[:,i]=Un
        Ud[:,i]=Udn
        Udd[:,i]=Uddn

    return U.T,Ud.T,Udd.T

In [17]:
def StarEnergy(self, others):
    retval = Newmark_initialize(ndof,a0,a1,M,C,K,NT,DI,VI) #TODO
    for other in others:
        retval += Change_in_Energy(self, others) #TODO
    return retval

# TODO: thumb through the library of SublimeText and find an estimate of local energy for constitutive model, E(X) = _Energy_in_the_Star_radius_(X) is an extrinsic estimate while \Psi(X) is an intrinstic measure of the ground state that relaxes on a reletively slow electrophysiological time scale, 1ms.

$\tau_\text{EM}=\tau_\text{Electromotive}\sim1\,ms \ll 25\mu s\sim \tau_\text{EP}=\tau_\text{Electrophysiological}$

# Generate an array of randomized steps
TODO: try improving ^this model by using a different step size in the normal direction defined by the stable configuration given.

In [9]:
#TODO: define a numpy function that defines a random unbiased step for each vertex
#TODO: find the (i) the continuum constitutive relations and (i) the spring constitutive relations
#TODO: define an energy density function as the sum of those of the neighboring triangles
#TODO: define several energy functionals
#TODO: define an accept/reject energy functional

X = mesh.vertices
#TODO: initialize displacement field to zero
U = mesh.vertices - X
def accept_step_query():
    pass

In [2]:
import trimesh, numpy as np, numba
from numba import njit,jit

In [8]:
#TODO: import mesh of sphere to define X
mesh_dir = nb_dir+"/Data/spherical_meshes/spherical_mesh_64.stl"
mesh = trimesh.load_mesh(mesh_dir)
X = mesh.vertices #\Omega_0 is the initially given configuration
x = X # set \Omega_{t=0} = \Omega_0
# mesh.show()

face_normals all zero, ignoring!


$$\Phi(X_i) \equiv \sum_{j=1}^{N_{X_i}}\Phi_j$$

Where $\Phi_j=$ is the energy density of the $j^{th}$ triangle neighboring the $i^{th}$ point $X_i$ located at $x(t)$ on the surface $\Omega_t$.

### define the energy of one triangle for the corotated linear model
from the notebook, "Elastic Model of RA", we have

In [ ]:
tet = np.stack((dm1,dm2,Am))
#half of the determinant of tet is the squared area of the triangle.
np.linalg.det(tet)/2==Am.dot(Am)
np.linalg.eigvals(tet)

# TODO: consider mapping between manifolds using scipy.optimize.minimize on some energy loss function.
methods str or callable, optional
Type of solver. Should be one of
- ‘Nelder-Mead’ (see here)
- ‘Powell’ (see here)
- ‘CG’ (see here)
- ‘BFGS’ (see here)
- ‘Newton-CG’ (see here)
- ‘L-BFGS-B’ (see here)
- ‘TNC’ (see here)
- ‘COBYLA’ (see here)
- ‘SLSQP’ (see here)
- ‘trust-constr’(see here)
- ‘dogleg’ (see here)
- ‘trust-ncg’ (see here)
- ‘trust-exact’ (see here)
- ‘trust-krylov’ (see here)
- custom - a callable object (added in version 0.14.0), see below for description.

In [40]:
from numba import vectorize
@vectorize('float32(float32, float32)')#, target='gpu')
def VectorAdd(a,b):
    return a + b
a = np.array(range(5),dtype=np.float32)


In [43]:
VectorAdd(a,a)

array([0., 2., 4., 6., 8.], dtype=float32)

# TODO: finish reading Augustin (2020).


See handwritten notes on Augustin (2020) 

which is a spacetime generalization of variational integrators that allows you to take different timesteps for different elements, which is important as slivers in the mesh often force you to take very small timesteps because of the CFL condition, and a global tensor product spacetime mesh would mean that the entire domain has to be advanced using that small timestep.

After today, I'll test that and see if I can reproduce the tests on an AVI on 2D flat space tessellated by triangles.  Once that works, I'll embed an example patient's atrium in 3D space, and tesselated it with triangles, and run the xgboost code on it, where I could smoothly track spiral tips on the atrial surface using a ray tracing projection onto the bezier surface generated by the patient's vertices as control knots.  



# TODO(after basic mvc pattern filled out): consider using a bezier surface to interpolate between the cardiac  surfaces that are updated at a time resolution of 1ms 

# TODO: write corotated linear strain energy functional
and then,

#TODO: create symplectic representation of the displacement and the rate of change of its displacement field, Q = (X, U)
#TODO: compute total energy for mesh H = (T+V)
#TODO: compute total action for mesh L = (T-V)

#TODO: minimize the action using scipy.optimize.minimize for any of the given methods
#TODO: if the first method doesn't work, try the next method.

In [44]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  

#automate the boring stuff
from IPython import utils
import time, os, sys, re
beep = lambda x: os.system("echo -n '\\a';sleep 0.2;" * x)
if not 'nb_dir' in globals():
    nb_dir = os.getcwd()
sys.path.append("../lib") 
from lib import *
sys.path.append("lib") 
from lib import *
from operari import *
from ProgressBar import *
# from mesh_ops import *

# the visualization tools involved here for triangular meshes is
import trimesh
import pyglet
from numba import njit, cuda
# from numba.typed import List
# import numba
import trimesh

#try using a scipy sparse matrix to speed up spring force evaluations
#TODO: speed up bigger meshes with pycuda's sparce matrices
from scipy.sparse import csr_matrix
import scipy.sparse as sparse

# from pyspark import SparkContext 
# sc = SparkContext(master="local[4]")
# print(sc)


%autocall 1
%load_ext autoreload
%autoreload 2

Automatic calling is: Smart
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
class BPSomeClass(object):
    r"""Describe the class"""
    def __init__(self, arg1, arg2):
        self.attr1 = arg1
        self.attr2 = arg2
    
    def attribute1(self):
        return self.attr1
bp_obj = BPSomeClass("a", 2.7182)
bp_obj.attribute1()

'a'

In [ ]:
#TODO: copy opencarp into myfolder
#TODO: test out opencarp
#TODO: pycharm a 3D viewer app template
#TODO: bash your way into python
#use scipy.optimize.minimize to map one frame to another frame with the corotated linear constitutive model
#optimize lib.model.energy_functional \sim lib.model.action_functional using gekko
